In [105]:
import numpy as np
from scipy.stats import poisson
import plotly.graph_objs as go
import plotly.offline as pyo
import unittest

# """
# Function to calculate the convolution of two arrays.
# 
# Args:
#     a (numpy.ndarray): The first array to be convolved.
#     b (numpy.ndarray): The second array to be convolved.
# 
# Returns:
#     numpy.ndarray: The convolution of the two input arrays.
# """
def convolve(a, b):
    
    # Initialize an empty array to store the result.
    c = np.array([])
    
    # Compute the convolution of the two arrays.
    for i in range(len(a)):
        # Get subsets of array expanded to the right.
        a_sub = a[0:i + 1].copy()
        b_sub = b[0:i + 1].copy()
        # Reverse b.
        b_rev = b_sub[::-1]
        # Compute the dot product of a and b_rev.
        c = np.append(c, np.dot(a_sub, b_rev))
    
    for i in range(1,len(a)):
        # Get subsets of array collapse from the right.
        a_sub = a[i:].copy()
        b_sub = b[i:].copy()
        # Reverse b.
        b_rev = b_sub[::-1]
        # Compute the dot product of a and b_rev.
        c = np.append(c, np.dot(a_sub, b_rev))
        
    return c


# """
# Function to convolve a distribution with itself n times.
# 
# Args:
#     a (numpy.ndarray): The distribution to be convolved.
#     n (int): The number of times to convolve the distribution with itself.
# 
# Returns:
#     numpy.ndarray: The convolution of the input distribution with itself n times.
# """
def convolve_n(a, n):
        
    # Initialize an empty array to store the result.
    c = np.array([])
    
    # If n is 0, return an array of zeros with length equal to the length of a, except for the first element which is 1.
    if n == 0:
        c = np.array(np.zeros(len(a)), dtype=np.float64)
        c[0] = 1
        return c
    
    # Convolve the distribution with itself n times.
    for i in range(n):
        # If this is the first iteration, set c equal to a.
        if i == 0:
            c = a
        # Otherwise, convolve c with a.
        else:
            c = np.convolve(c, a)
            
    return c
# 
# """
# Function to create an array of zero arrays according to a given shape array.
# 
# Args:
#      num_zeros (numpy.ndarray): The shape array.
#      l (int): The length of the zeros array.
#  
# Returns:
#      numpy.ndarray: The convolution of the input distribution with itself n times.
# """

def zero_arrays(num_zeros, l):
    result = []
    for n in num_zeros:
        zeros = np.zeros(l)
        result.append([zeros] * n)
    return result

zero_arrays(np.array([1, 0, 3]), 4)

def calc_distr_limit(l):
    return int(max(l+4*l**0.5, 100))


class TestConvolve(unittest.TestCase):
    
    def test_convolve(self):
        a = np.array([
            0.4456796414,
            0.160623141,
            0.137676978,
            0.1032577335])

        b = np.array([
            0.006737946999,
            0.033689735,
            0.08422433749,
            0.1403738958])

        expected_output = np.convolve(a, b)
        
        self.assertTrue(np.allclose(convolve(a, b), expected_output))
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [106]:
zero_arrays(np.array([1, 0, 3]), 4)

[[array([0., 0., 0., 0.])],
 [],
 [array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([0., 0., 0., 0.])]]

A schedule with T intervals can have T states. A state of a schedule at interval T is defined by $p^-_t(i)$, the amount of work left at the end of the state at interval $t-1$ and $v_t(i)$, the arriving amount of work in interval $t$.

In [196]:

"""
A schedule class with a constructor and a method to calculate the system states.
"""
class Schedule:
    def __init__(self, x, d, s):
            # Initialize the parameters dictionary with x, d, and s.
            self.parameters = dict({'x': x, 'd': d, 's': s})
            # Initialize the system dictionary with p_min and v.
            self.system = dict({
                'p_min': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64),
                'v': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64)
            })
            # Set the initial state to 0.
            self.state = 0
            # Set the first element of p_min in the initial state to 1.
            self.system['p_min'][self.state][0] = 1
            # Set the first row of v to s.
            self.system['v'][self.state] = convolve_n(self.parameters['s'].copy(), self.parameters['x'][self.state])[:(len(self.parameters['s']))]
            # Set p_plus to v.
            self.system['p_plus'] = self.system['v'].copy()
            # Initialize array of arrays for saving waiting times distributions per patient
            self.system['w'] = zero_arrays(self.parameters['x'].copy(), len(self.parameters['s']))
            if(self.parameters['x'][self.state] > 0): self.system['w'][self.state][0] = self.system['p_min'][self.state]
            # Initializ array for saving total expected waiting times per state
            self.system['ew'] = np.zeros(len(self.parameters['x']), dtype=np.float64)
        
    def calculate_system_states(self, until = 1):

            while self.state < until:
                """Set state to 1, because state 0 has already been calculated in the constructor.
                TO CHECK: WHAT HAPPENS IF SOMETHING CHANGES IN STATE 0?
                """
                
                self.state += 1
                
                """The probability that the amount of work left in the system equals zero just before state t starts is the probablity that the total amount work in state t-1 was less than or equal to the interval length d."""
                
                self.system['p_min'][self.state][0] = np.sum(self.system['p_plus'][self.state-1][:(self.parameters['d'] + 1)])
                
                """The probability that the amount of work left in the system equals i just before state t starts is the probablity that the total amount work in state t-1 exceeded the interval length d with amount i."""
                
                self.system['p_min'][self.state][1:(-1*self.parameters['d'])] = self.system['p_plus'][self.state-1][(self.parameters['d'] + 1):]
                
                """Service time distribution are calculated as n times convolutions, where n is the number of clients scheduled. The resulting vector is truncated to the length of the service time distribution."""
                self.system['v'][self.state] = convolve_n(self.parameters['s'].copy(), self.parameters['x'][self.state])[:(len(self.parameters['s']))]
                
                if(self.parameters['x'][self.state] > 0):
                     self.system['w'][self.state][0] = self.system['p_min'][self.state].copy()
                     for i in range(1, self.parameters['x'][self.state]):
                            self.system['w'][self.state][i] = np.convolve(self.system['w'][self.state][i-1], self.parameters['s'])[:(len(self.parameters['s']))]

                """The distribution of waiting times of the first patient in state t equals p_min. 
                The distribution of waiting times of the second patient in state t equals the convolution of the distribution 
                of waiting times of the first patient in state t and the service time distribution. 
                The resulting vector is truncated to the length of the service time distribution."""
                
                """The probablitity that the amount of work left in the system equals i just before state t ends equals 
                the convolution of the probability that the amount of work left in the system equals i just before state t starts 
                and the service time distribution. The resulting vector is truncated to the length of the service time distribution."""
                self.system['p_plus'][self.state] = self.system['p_min'][self.state].copy() if self.parameters['x'][self.state] == 0 else np.convolve(self.system['w'][self.state][-1], self.parameters['s'][self.state])[:(len(self.parameters['s']))]
                # self.system['p_plus'][self.state] = np.convolve(self.system['p_min'][self.state], self.system['v'][self.state])[:(len(self.parameters['s']))]
                    
    
    def calculate_wait_times(self, until = 1):
    # Initialize time counter
      t = 0
      # Calculate the expected waiting time for each time step
      for t, w in enumerate(self.system['w']):
          ew = 0
          for i in w:
              # Calculate the weighted sum of the waiting time distribution
              a = range(len(i))
              b = i
              wt = np.dot(a, b)
              ew += wt
          # Store the expected waiting time for the current time step
          self.system['ew'][t] = ew
        
        
    def visualize_p_plus(self):
        """
        Visualize p_plus in the last calculated state.
        """
        
        trace = go.Scatter(
            x=list(range(len(self.system['p_plus'][self.state]))),
            y=self.system['p_plus'][self.state],
            mode='lines'
        )
        
        data = [trace]
        
        layout = go.Layout(
            title=f'p_plus in state {self.state}',
            xaxis=dict(title='Total amount of work in the system'),
            yaxis=dict(title='Probability')
        )
        
        fig = go.Figure(data=data, layout=layout)
        pyo.iplot(fig)
        
    def __str__(self):
        """
        Print all probabilities
        """
        return("p_min = % s \nw = % s \nv = % s \np_plus = % s \new = % s" % (self.system['p_min'], self.system['w'] , self.system['v'], self.system['p_plus'], self.system['ew']))

The code above defines a `Schedule` class that has a constructor and a method to calculate the system states. The constructor initializes the class with three parameters: `x`, `d`, and `s`. These parameters are stored in a dictionary called `parameters`. The constructor also initializes two dictionaries called `system` and `state`. The `system` dictionary contains two numpy arrays called `p_min` and `v`. The `state` dictionary contains a single integer value that is initialized to 0.

The `calculate_system_states` method takes an optional parameter called `until` that defaults to 1. This method calculates the system states until the specified `until` value is reached. The method uses a while loop to iterate over the states until the `until` value is reached. The first step in the loop is to increment the `state` value by 1. 

The next step is to calculate the probability that the amount of work left in the system equals zero just before state `t` starts. This is done by summing the probability of all the previous states that have a total amount of work less than or equal to the interval length `d`. The result is stored in the `p_min` array.

The next step is to calculate the probability that the amount of work left in the system equals `i` just before state `t` starts. This is done by taking the probability of all the previous states that have a total amount of work greater than the interval length `d` with an amount of `i`. The result is stored in the `p_min` array.

The next step is to calculate the service time distribution. This is done by taking the convolution of the service time distribution `s` with the number of clients scheduled `x` for the current state. The result is stored in the `v` array.

The final step is to calculate the probability that the amount of work left in the system equals `i` just before state `t` ends. This is done by taking the convolution of the probability that the amount of work left in the system equals `i` just before state `t` starts and the service time distribution. The result is stored in the `p_plus` array.

Overall, this code is well-organized and easy to read. However, there are a few ways to improve the code's readability and performance. For example, the `parameters` dictionary could be replaced with individual variables to make the code more readable. Additionally, the `convolve_n` function used in the code could be replaced with a more efficient implementation to improve performance. Finally, the comments in the code could be improved to provide more context and explanation for the calculations being performed.

In [197]:
def run_schedule(x, d, s, u=1):
  schedule = Schedule(x=x, d=d, s=s)
  schedule.calculate_system_states(until=u)
  schedule.calculate_wait_times(until=u)
  print(schedule)
  return(schedule)

In [198]:
"""Simple test case"""

x = np.array([1, 0, 3, 1, 1], dtype=np.int64)
d = 3
s=np.array(
    [0.1, 0.2, 0.3, 0.2, 0.15, 0.05, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float64)
u = 4

run_schedule(x, d, s, u)

p_min = [[1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [8.000e-01 1.500e-01 5.000e-02 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [9.300e-02 6.000e-02 5.700e-02 4.500e-02 2.700e-02 1.275e-02 4.500e-03
  7.500e-04 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [5.100e-02 5.400e-03 2.550e-03 9.000e-04 1.500e-04 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]] 
w = [[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])], [], [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0.1 , 0.2 , 0.3 , 0.2 , 0.15, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  ]), array([0.01  , 0.04  , 

In [123]:
""""Test case with validation in spreadsheet
url: https://docs.google.com/spreadsheets/d/1_l9bMqEfLT2-TpZz3MrDFIid30ZsywTVH8Lzc5uHrGw/edit?usp=sharing"""
x = np.array([1, 0, 1, 0, 1, 0, 1], dtype=np.int64)
d = 3
l = 6
limit = calc_distr_limit(l * x.sum())+1
service_times = range(0, limit)
s = np.array([poisson.pmf(i, l) for i in service_times], dtype=np.float64)
u = 2
  
run_schedule(x, d, s, u)

2 0
p_min = [[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e

In [113]:
""""Test case with more than 1 clients in the system at the same time
url: https://docs.google.com/spreadsheets/d/1_l9bMqEfLT2-TpZz3MrDFIid30ZsywTVH8Lzc5uHrGw/edit?usp=sharing"""
x = np.array([2, 0, 1, 0, 1, 1, 1], dtype=np.int64)
d = 5
l = 6
limit = calc_distr_limit(l * x.sum())+1
service_times = range(0, limit)
s = np.array([poisson.pmf(i, l) for i in service_times], dtype=np.float64)
u = 4
  
schedule = run_schedule(x, d, s, u)



2 0
4 0
p_min = [[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000

In [114]:
schedule.visualize_p_plus()